In [1]:
from datetime import date
import json
import os
import re

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import dateutil.parser

In [ ]:
%%time
df = pd.read_csv('../data/scotus_file', usecols = ['author', 'html'])

In [ ]:
print(len(df))
df.dropna(how="any", inplace=True)
print(len(df))

In [ ]:
%%time
def parse_html(raw_html):
    return BeautifulSoup(raw_html, "html.parser").text
df['text'] = df.apply(lambda row: parse_html(row['html']), axis=1)

In [2]:
%%time
df = pd.read_csv('../data/parsed.csv')

CPU times: user 9.25 s, sys: 1.18 s, total: 10.4 s
Wall time: 10.4 s


In [3]:
%%time
names = 10000 * [None] # names[i] = (first, middle, last)
PEOPLE_ROOT = "../data/people"
for file_name in os.listdir(PEOPLE_ROOT):
    author_idx = int(file_name.split('.')[0])
    with open(os.path.join(PEOPLE_ROOT, file_name)) as f:
        data = json.load(f)
        names[author_idx] = (data['name_first'], data['name_middle'], data['name_last'])
names[0:5]

CPU times: user 464 ms, sys: 96.3 ms, total: 560 ms
Wall time: 559 ms


In [4]:
def get_idx(author_url): # returns int
    return int(author_url.split('/')[-2])
get_idx(df['author'][df.index[0]])

26

In [5]:
%%time
fml = df.apply(lambda row: names[get_idx(row['author'])], axis = 1)
first, middle, last = map(list, zip(*fml))
last[0:5]

CPU times: user 346 ms, sys: 2.7 ms, total: 349 ms
Wall time: 348 ms


In [6]:
df['first'] = first
df['middle'] = middle
df['last'] = last

In [7]:
%%time
# See https://bear.im/code/parsedatetime/docs/index.html for additional documentation
pattern0 = re.compile("Decided.*? [0-9]{4}")
prefixes = ["Decided", "Argued", "Filed", "Submitted", "Devided", "Ex parte BELT\.",
            "Decree entered", "Denied"]
pattern1 = re.compile("(" + "|".join(prefixes) +").*? [0-9]{4}")
pattern2 = re.compile("Term.* [0-9]{4}")
pattern3 = re.compile("\[.*[0-9], [0-9]{4}\]")
months = ["January", "February", "March", "April", "May",
          "June", "July", "August", "September", "October",
          "November", "December",
          "Jan\.", "Feb\.", "Mar\.", "Apr\.",
          "Jun\.", "Jul\.", "Aug\.", "Sep\.",
          "Oct\.", "Nov\.", "Dec\."]
pattern4 = re.compile("(" + "|".join(months) +") ?[0-9]{0,2},  ?[0-9]{4}")

def get_date(text):
    search_res = pattern0.search(text, 0, 3000)
    if search_res:
        raw = search_res.group()[-4:]
        raw = "May 9, " + raw
        try:
            return dateutil.parser.parse(raw).date()
        except:
            return None
    
    search_res = pattern1.search(text, 0, 3000)
    if search_res:
        raw = search_res.group()[-4:]
        raw = "May 9, " + raw
        try:
            return dateutil.parser.parse(raw).date()
        except:
            return None
    
    search_res = pattern2.search(text, 0, 2000)
    if search_res:
        raw = search_res.group()[-4:]
        raw = "May 9, " + raw
        try:
            return dateutil.parser.parse(raw).date()
        except:
            return None
    
    search_res = pattern3.search(text, 0, 2000)
    if search_res:
        raw = search_res.group()[1:-1]
        try:
            return dateutil.parser.parse(raw).date()
        except:
            return None
        
    search_res = pattern4.search(text, 0, 2000)
    if search_res:
        raw = search_res.group()[-4:]
        raw = "May 9, " + raw
        try:
            return dateutil.parser.parse(raw).date()
        except:
            return None
    
    """
    lines = text[0 : 500].split('\n')
    for line in lines:
        line = line.replace('[', '')
        line = line.replace(']', '')
        try:
            res = dateutil.parser.parse(line).date()
            return res
        except:
            pass
    """
    
    return None

df['date'] = df.apply(lambda row: get_date(row['text']), axis=1)
#print(get_date(df['text'][df.index[21099]]))

CPU times: user 1.92 s, sys: 11.2 ms, total: 1.93 s
Wall time: 1.93 s


In [8]:
print(len(df[pd.isna(df.date)]), len(df))

1 21337


In [9]:
df[pd.isna(df.date)][0:5]

,Unnamed: 0,author,html,text,first,middle,last,date
14070,42197,http://www.courtlistener.com/api/rest/v3/peopl...,"<p class=""case_cite"">103 U.S. 370</p>\n <p ...",103 U.S. 370\n103 U.S. 370\n26 L.Ed. 567\nNEAL...,John,Marshall,Harlan,None


In [10]:
#print(df['date'][14070])
#print(df['text'][14070])

In [11]:
print(len(df))
df.dropna(how="any", inplace=True)
print(len(df))

21337
21336


In [13]:
%%time
df.to_csv('../data/parsedv5.csv')

CPU times: user 30.3 s, sys: 1.04 s, total: 31.3 s
Wall time: 31.9 s


In [ ]:
sorted(df['date'])[-10:]

In [ ]:
list(filter(lambda iy: iy[1].year <=1792, enumerate(df['date'])))

In [ ]:
print(df['date'][df.index[20263]])
print(df['text'][df.index[20263]])

In [ ]:
for i, t in enumerate(df['last']):
    if t == 'Roberts':
        print(i)

In [ ]:
df['text'][df.index[7290]]

In [ ]:
df.index[7290]

In [ ]:
dateutil.parser.parse("[May 17, 1999]")